In [1]:
import os
import sys
import json
import time
import requests

from dotenv import load_dotenv

def setup_env():
    
    env_path = os.path.join(os.getcwd(), '../.env')

    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        
        print(f"Loaded environment variables from: \033[94m{env_path}\033[0m")
    else:
            print("\033[91mError: .env file not found. Please create one with your OPENAI_API_KEY.\033[0m")
            sys.exit(1)

setup_env()

Loaded environment variables from: /home/ras/0.agent_ai_ws/src/learn_rag_and_agent/learn_rag_and_agent/../.env


In [2]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores.faiss import FAISS

# OpenAI 임베딩을 사용하여 기본 임베딩 설정
embedding = OpenAIEmbeddings()

# 로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

# 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embedding,
    document_embedding_cache=store,
    namespace=embedding.model,  # 기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

In [3]:
# store에서 키들을 순차적으로 가져옵니다.
list(store.yield_keys())

['text-embedding-ada-002cc824f84-d691-544f-9d9c-ca7e45470bb2',
 'text-embedding-ada-0029db9e1cd-62d8-50fc-94f4-24bef3cacaf5',
 'text-embedding-ada-00241e7391b-b68f-5e9f-bb07-3609bb83c3e2',
 'llm_cache.db',
 'text-embedding-ada-0027494a7c8-3399-52a1-85ef-f4d0a563d31f',
 'text-embedding-ada-0020fd71f95-1342-512d-9d5b-3e3ab3c6bbe0',
 'text-embedding-ada-0022112b0ec-6ade-59c9-b09c-755b33c3d32c',
 'text-embedding-ada-00274ae75af-9058-555e-aefa-082f0b4e0560',
 'hub/models--beomi--llama-2-ko-7b/blobs/6665b3487fa86ca6701a1921b710117e0f78d604',
 'hub/models--beomi--llama-2-ko-7b/blobs/89b127dcb3c85427a3c9224224c60f344d45eacdb729e4352a42224bdbed76ba',
 'hub/models--beomi--llama-2-ko-7b/blobs/342d75efdb4bffcd94f59df101808425c2c7a95e',
 'hub/models--beomi--llama-2-ko-7b/blobs/f6986fbe739e40bcf83ce679c687b98aa9dce2b0621604e70e80f76ad5e13ada',
 'hub/models--beomi--llama-2-ko-7b/blobs/824354aba3909363d96c7958c4d1ef2d46cfd0150ad7dbf4a12ae7e6388dc8ab',
 'hub/models--beomi--llama-2-ko-7b/blobs/ceaedeb4b

In [28]:
from langchain.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

# 문서 로드
raw_documents = TextLoader("kumdori_info.txt").load()
# 문자 단위로 텍스트 분할 설정
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=200)
# 문서 분할
documents = text_splitter.split_documents(raw_documents)

In [29]:
# 코드 실행 시간을 측정합니다.
%time db = FAISS.from_documents(documents, cached_embedder)  # 문서로부터 FAISS 데이터베이스 생성

CPU times: user 27.2 ms, sys: 447 μs, total: 27.7 ms
Wall time: 392 ms


In [37]:
# 캐싱된 임베딩을 사용하여 FAISS 데이터베이스 생성
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 125 μs, sys: 6.99 ms, total: 7.12 ms
Wall time: 5.72 ms


In [39]:
print("FAISS 데이터베이스가 성공적으로 생성되었습니다.")

# k 설정
retriever = db2.as_retriever(
    search_kwargs={"k": 1},
    )

# 관련 문서를 검색
query = "서울 가면 뭐할 수 있나요"

for doc in retriever.invoke(query):
    print(doc.page_content)
    print("=========================================================")

FAISS 데이터베이스가 성공적으로 생성되었습니다.
#### B. 대전 0시 축제 (미래존 부스)
* **축제 특징:** 새해를 맞이하는 특별한 순간을 기념하기 위해 0시에 다양한 행사와 불꽃놀이를 진행합니다.
* **추천 활동:** 미래존 부스에서의 인터랙티브 체험, 불꽃놀이 (대전 엑스포 다리 근처에서 축제 마지막에 진행됨).
* **주변 맛집:** '명품대패' (24시간 고기집), 수제버거, 과일 주스 전문점, '비타민을 담다' (과일 주스 전문점), 배스킨라빈스, '몽플' (수플레/음료).

#### C. 대전 드림 아레나 (e스포츠 경기장)
* **기능:** e스포츠 대회 및 이벤트가 열리는 공간입니다.
* **규모:** 총 500석 규모의 주경기장과 50석 규모의 보조경기장을 갖춥니다.
* **시설:** 최신 음향 및 조명 시설, 쾌적한 관람 환경을 제공합니다.
* **내부 시설 (예시):** '비타민을 담다' (과일, 주스 전문점), '더 리더스 부동산'.


In [40]:
# 관련 문서를 검색
query = "비타민을 담다가 무엇인가요"

for doc in retriever.invoke(query):
    print(doc.page_content)
    print("=========================================================")

#### B. 대전 0시 축제 (미래존 부스)
* **축제 특징:** 새해를 맞이하는 특별한 순간을 기념하기 위해 0시에 다양한 행사와 불꽃놀이를 진행합니다.
* **추천 활동:** 미래존 부스에서의 인터랙티브 체험, 불꽃놀이 (대전 엑스포 다리 근처에서 축제 마지막에 진행됨).
* **주변 맛집:** '명품대패' (24시간 고기집), 수제버거, 과일 주스 전문점, '비타민을 담다' (과일 주스 전문점), 배스킨라빈스, '몽플' (수플레/음료).

#### C. 대전 드림 아레나 (e스포츠 경기장)
* **기능:** e스포츠 대회 및 이벤트가 열리는 공간입니다.
* **규모:** 총 500석 규모의 주경기장과 50석 규모의 보조경기장을 갖춥니다.
* **시설:** 최신 음향 및 조명 시설, 쾌적한 관람 환경을 제공합니다.
* **내부 시설 (예시):** '비타민을 담다' (과일, 주스 전문점), '더 리더스 부동산'.


In [41]:
query = "수소 트램에 대해 설명해주세요"

for doc in retriever.invoke(query):
    print(doc.page_content)
    print("=========================================================")

### 4. 대전시 및 기타 일반 지식
* **대전시 특징:** 과학과 기술의 도시로 유명합니다.
* **대전 유명 인물:** 이장우 대전광역시장 (2022년 6월 1일 취임).
* **주요 관광지:** 대전 엑스포 과학공원, 한밭수목원, 대전문화재단 패밀리 테마파크.
* **야구팀:** 한화 이글스 (대전 지역 프로 야구팀, 홈구장: 대전 한화생명이글스파크).
* **음식 추천 (일반):** '한방삼계탕', 성심당의 '앙버터'와 '모카빵'.
* **기타 지식:**
    * **수소 트램:** 수소 연료전지를 이용해 전력을 생성하여 운행되는 친환경 트램입니다.
    * **나비:** 곤충의 한 종류로, 꽃가루를 옮기는 중요한 역할(pollinator)을 합니다.
    * **크롬:** 구글이 개발한 웹 브라우저입니다.
    * **사랑:** 서로를 아끼고 이해하는 감정입니다.


In [43]:
query = "한화이글스에 대해 설명해주세요"

for doc in retriever.invoke(query):
    print(doc.page_content)
    print("=========================================================")

### 4. 대전시 및 기타 일반 지식
* **대전시 특징:** 과학과 기술의 도시로 유명합니다.
* **대전 유명 인물:** 이장우 대전광역시장 (2022년 6월 1일 취임).
* **주요 관광지:** 대전 엑스포 과학공원, 한밭수목원, 대전문화재단 패밀리 테마파크.
* **야구팀:** 한화 이글스 (대전 지역 프로 야구팀, 홈구장: 대전 한화생명이글스파크).
* **음식 추천 (일반):** '한방삼계탕', 성심당의 '앙버터'와 '모카빵'.
* **기타 지식:**
    * **수소 트램:** 수소 연료전지를 이용해 전력을 생성하여 운행되는 친환경 트램입니다.
    * **나비:** 곤충의 한 종류로, 꽃가루를 옮기는 중요한 역할(pollinator)을 합니다.
    * **크롬:** 구글이 개발한 웹 브라우저입니다.
    * **사랑:** 서로를 아끼고 이해하는 감정입니다.
